## Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score
from tensorflow.keras.layers import TimeDistributed, Dense
import gc
import time

In [2]:
USE_CPU_ONLY = True 

if USE_CPU_ONLY:
    print("\n[STABILITY MODE] Disabling GPU to prevent Metal compilation hangs...")
    tf.config.set_visible_devices([], 'GPU')
else:
    # Re-adding the memory growth code for when GPU is enabled
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("[GPU MODE] Memory growth enabled.")
        except RuntimeError as e:
            print(f"Memory growth setting failed: {e}")


[STABILITY MODE] Disabling GPU to prevent Metal compilation hangs...


## Read heldout processed data

In [3]:
heldout_df = pd.read_csv('/Users/adityaponnada/Downloads/time_study_data/general_heldout_scaled.csv')  # Replace with actual path
heldout_df.head()

,participant_id,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,closeness_to_sleep_time,...,wake_day_part_2.0,wake_day_part_3.0,mi_in_battery_saver_mode,mi_charging_status,mi_dist_from_home,mi_is_phone_locked,mi_last_phone_usage,mi_closeness_to_sleep_time,mi_closeness_to_wake_time,mi_mims_5min
0,animateshowerclothes@timestudy_com,1,0,1.0,0.0,1,-40.137687,1.0,-0.589123,1.842273,...,0,0,0,0,0,1,1,0,0,0
1,animateshowerclothes@timestudy_com,0,0,1.0,1.0,1,-40.136734,1.0,-0.589123,1.739076,...,0,0,0,0,0,1,1,0,0,0
2,animateshowerclothes@timestudy_com,0,0,0.0,1.0,1,-40.137375,1.0,-0.589123,1.694101,...,0,0,0,0,0,1,1,0,0,0
3,animateshowerclothes@timestudy_com,0,0,0.0,1.0,1,-40.133405,1.0,-0.589123,1.649200,...,0,0,0,0,0,1,1,0,0,0
4,animateshowerclothes@timestudy_com,0,0,0.0,1.0,1,-40.132982,1.0,-0.589123,1.487557,...,0,0,0,0,0,1,1,0,0,0


In [4]:
def max_observations_per_participant(df, participant_col='participant_id'):
    """Return the maximum number of observations any participant has in `df`."""
    import pandas as pd
    if df is None or participant_col not in df.columns:
        raise ValueError('DataFrame must contain the participant column')
    counts = df[participant_col].value_counts()
    if counts.empty:
        return 0
    return int(counts.max())

# Example usage
max_obs = max_observations_per_participant(heldout_df)
print('max observations per participant:', max_obs)
max_obs

max observations per participant: 15169


15169

In [5]:
# Print number of columns excluding participant_id and outcome
excluded = {'participant_id', 'outcome'}
feature_cols = [c for c in heldout_df.columns if c not in excluded]
print('number of feature columns (excluding participant_id and outcome):', len(feature_cols))
# optional: show first few feature column names
print('example feature columns:', feature_cols[:10])

number of feature columns (excluding participant_id and outcome): 40
example feature columns: ['is_weekend', 'in_battery_saver_mode', 'charging_status', 'screen_on', 'dist_from_home', 'is_phone_locked', 'last_phone_usage', 'closeness_to_sleep_time', 'closeness_to_wake_time', 'mims_5min']


## Define constants

In [6]:
L_CHUNK = 3967
NUM_CHUNKS = 4
MAX_TIME_SLOTS = max_obs 
NUM_FEATURES = len(feature_cols) 
SENTINEL_VALUE = 999.0
LEARNING_RATE = 1e-4
FINE_TUNE_EPOCHS = 10
CLASS_WEIGHTS = tf.constant([0.8, 0.2], dtype=tf.float32)

## Split user data

In [7]:
def split_user_data_temporally(df, split_ratio=0.1):
    """
    Splits each user's data into a 10% morning snapshot (train) 
    and 90% future window (test) before padding.
    """
    train_df_list = []
    test_df_list = []
    
    grouped = df.groupby('participant_id')
    for p_id, group in grouped:
        n_steps = len(group)
        split_idx = int(n_steps * split_ratio)
        
        train_df_list.append(group.iloc[:split_idx])
        test_df_list.append(group.iloc[split_idx:])
        
    # Combine into two distinct DataFrames
    train_df = pd.concat(train_df_list, ignore_index=True)
    test_df = pd.concat(test_df_list, ignore_index=True)
    
    return train_df, test_df

In [8]:
train_df, test_df = split_user_data_temporally(heldout_df, split_ratio=0.1)

## Prepare tensors

In [9]:
def prep_tensors_from_df(df, target_chunks):
    """
    Converts a pre-scaled DataFrame into 4D tensors.
    target_chunks: 1 for training (10%), 4 for testing (90%).
    """
    grouped = df.groupby('participant_id')
    X_list, Y_list, p_ids = [], [], []
    
    for p_id, group in grouped:
        p_ids.append(p_id)
        X_seq = group.drop(columns=['participant_id', 'outcome']).values 
        Y_seq = group['outcome'].values.astype('float32').reshape(-1, 1)
        X_list.append(X_seq)
        Y_list.append(Y_seq)

    # Calculate exact padding required for the chunk structure
    required_len = target_chunks * L_CHUNK

    X_p = pad_sequences(X_list, maxlen=required_len, padding='post', value=SENTINEL_VALUE, dtype='float32')
    Y_p = pad_sequences(Y_list, maxlen=required_len, padding='post', value=SENTINEL_VALUE, dtype='float32')
    
    X_4d = X_p.reshape(len(p_ids), target_chunks, L_CHUNK, NUM_FEATURES)
    Y_4d = Y_p.reshape(len(p_ids), target_chunks, L_CHUNK, 1)
    
    return tf.cast(X_4d, tf.float32), tf.cast(Y_4d, tf.float32), p_ids

In [13]:
# Training tensor uses 1 chunk (the 10% snapshot)
X_train_4d, Y_train_4d, p_ids = prep_tensors_from_df(train_df, target_chunks=1)

# Testing tensor uses 4 chunks (the 90% future window)
X_test_4d, Y_test_4d, _ = prep_tensors_from_df(test_df, target_chunks=4)

## Custom metric and loss functions

In [10]:
def mask_generator_fn(x):
    return tf.cast(tf.not_equal(x[:, :, :1], SENTINEL_VALUE), tf.float32)

def optimized_loss_fn(y_true, y_pred):
    """Original loss function required for model loading."""
    import tensorflow as tf
    mask = tf.cast(tf.not_equal(y_true, 999.0), tf.float32)
    y_p = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
    bce = - (y_true * tf.math.log(y_p) + (1.0 - y_true) * tf.math.log(1.0 - y_p))
    y_true_int = tf.cast(tf.squeeze(y_true, axis=-1), tf.int32)
    y_true_clipped = tf.clip_by_value(y_true_int, 0, 1)
    c_weights = tf.constant([0.8, 0.2], dtype=tf.float32)
    weights = tf.gather(c_weights, y_true_clipped)
    weights = tf.expand_dims(weights, axis=-1)
    return tf.reduce_sum(bce * weights * mask) / (tf.reduce_sum(mask) + 1e-7)

def optimized_f1_class0(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, SENTINEL_VALUE), tf.float32)
    y_t, y_p = (1.0 - y_true) * mask, (1.0 - tf.math.round(y_pred)) * mask
    tp = tf.reduce_sum(y_t * y_p)
    fp = tf.reduce_sum((1.0 - y_t) * y_p * mask)
    fn = tf.reduce_sum(y_t * (1.0 - y_p) * mask)
    p, r = tp / (tp + fp + 1e-7), tp / (tp + fn + 1e-7)
    return 2 * ((p * r) / (p + r + 1e-7))

def get_personalized_loss(w0, w1):
    user_weights = tf.constant([w0, w1], dtype=tf.float32)
    def loss_fn(y_true, y_pred):
        mask = tf.cast(tf.not_equal(y_true, SENTINEL_VALUE), tf.float32)
        y_p = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
        bce = - (y_true * tf.math.log(y_p) + (1.0 - y_true) * tf.math.log(1.0 - y_p))
        y_true_int = tf.cast(tf.squeeze(y_true, axis=-1), tf.int32)
        weights = tf.gather(user_weights, tf.clip_by_value(y_true_int, 0, 1))
        return tf.reduce_sum(bce * tf.expand_dims(weights, -1) * mask) / (tf.reduce_sum(mask) + 1e-7)
    return loss_fn

## Fine tuning loops

In [14]:
def run_personalization_study(model_path, X_train_4d, Y_train_4d, X_test_4d, Y_test_4d, p_ids):
    """
    Optimized: Loads the model ONCE and resets weights in-memory for each user.
    Skips base model comparison as requested.
    """
    results = []
    
    # PHASE 0: LOAD MODEL ONCE
    print(f"\nLoading base model from {model_path}...")
    model = load_model(model_path, custom_objects={
        'optimized_loss_fn': optimized_loss_fn,
        'optimized_f1_class0': optimized_f1_class0,
        'mask_generator_fn': mask_generator_fn,
        'tf': tf
    }, compile=False)
    
    # Store initial pre-trained weights for resets
    global_weights = model.get_weights()
    
    # Freeze convolutional layers (Done once)
    for layer in model.layers:
        is_trainable = isinstance(layer, (TimeDistributed, Dense)) or \
                       any(k in layer.name.lower() for k in ['dense', 'output', 'time_distributed'])
        layer.trainable = is_trainable

    print(f"Starting optimized loop for {len(p_ids)} participants...")

    for i in range(len(p_ids)):
        start_user_time = time.time()
        
        # 1. RESET MODEL WEIGHTS (Instant in-memory operation)
        model.set_weights(global_weights)
        
        # 2. Individual Imbalance Weights from 10% morning
        y_train_flat = Y_train_4d[i].numpy().flatten()
        y_clean = y_train_flat[y_train_flat != SENTINEL_VALUE]
        
        if len(y_clean) == 0:
            print(f"[{i+1}/{len(p_ids)}] Skipping {p_ids[i]}: No valid training data.")
            continue

        c0, c1 = np.sum(y_clean == 0), np.sum(y_clean == 1)
        w0 = (len(y_clean) / (2 * (c0 + 1e-6))) if c0 > 0 else 1.0
        w1 = (len(y_clean) / (2 * (c1 + 1e-6))) if c1 > 0 else 1.0
        nw0, nw1 = w0 / (w0 + w1), w1 / (w0 + w1)
        
        # 3. Compile and Fine-Tune (10% slice)
        model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE), 
                      loss=get_personalized_loss(nw0, nw1))
        
        # Train on User's 10% (3D slice: target_chunks=1, L=3967, D=40)
        model.fit(X_train_4d[i], Y_train_4d[i], epochs=FINE_TUNE_EPOCHS, batch_size=1, verbose=0)
        
        # 4. Evaluate Personalized Score on 90% anchor
        y_true_test = Y_test_4d[i].numpy().flatten()
        mask_test = y_true_test != SENTINEL_VALUE
        
        # Predict on User's 90% (3D slice: target_chunks=4, L=3967, D=40)
        ft_probs = model.predict(X_test_4d[i], batch_size=1, verbose=0).flatten()
        
        y_true_real = y_true_test[mask_test]
        ft_f1 = f1_score(y_true_real, (ft_probs[mask_test] > 0.5).astype(int), 
                         pos_label=0, zero_division=0)
        
        user_duration = time.time() - start_user_time
        results.append({'participant_id': p_ids[i], 'ft_f1': ft_f1})
        print(f"[{i+1}/{len(p_ids)}] {p_ids[i]} | FT F1: {ft_f1:.4f} | Time: {user_duration:.1f}s")
        
        # Standard garbage collection
        gc.collect()

    return pd.DataFrame(results)

## Run tests

In [16]:
s1_results = run_personalization_study('best_model_safe.h5', X_train_4d, Y_train_4d, X_test_4d, Y_test_4d, p_ids)


Loading base model from best_model_safe.h5...
Starting optimized loop for 36 participants...


KeyboardInterrupt: 